# Finger tapping experiment: data setup

This notebook contains the workflow required to convert raw T1w and multi-echo fMRI data into the formats, types and structures necessary for the main data processing pipeline. Thus, getting data ready to be processed.

## Data

Per subject, all of the following data are collected during one scan session (all functional scans are multi-echo EPI):

| Nr | Name  | Scan Type | Description | Format |
| :--- | :--- | :--- | :--- | :--- |
| 1 | T1w | Anatomical | Standard T1-weighted sequence | NIfTI |
| 3 | run1_BOLD_task1 | Functional | Motor - finger tapping | PAR/REC, XML/REC, DICOM |

**Export data instructions:**
    - Non-dicom only
    - use file-names
    - all available formats


## Data setup goals

For each dataset (i.e. for each subject) we have to:

1. Move all files into a machine readable directory structure
2. Rename all image files in this directory structure such that BIDS tags are findable
3. Convert data to BIDS (using `bidsify` to convert the image data, which includes conversion of PAR/REC to NIfTI using `dcm2niix`)

## Required packages / software

- numpy, matplotlib, etc
- bidsify: https://github.com/NILAB-UvA/bidsify/tree/master/bidsify

In [5]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [9]:
import os
import shutil
import glob
import matplotlib.pyplot as plt
import numpy as np
import nibabel as nib
import bidsify
import pydeface
import pandas as pd
import json
from bids_validator import BIDSValidator
from convert_eprime.convert import text_to_csv
from scanphyslog2bids.core import PhilipsPhysioLog

## Step 0: initialize variables, directories, filenames, etc


In [14]:
# Dependent on structure and filenames in current data on external drive
data_dir = '/Users/jheunis/Desktop/KempCourse_data/'
org_data_dir = '/Users/jheunis/Desktop/KempCourse_data_organised/'
new_org_data_dir = org_data_dir + 'new/'
bids_data_dir = '/Users/jheunis/Desktop/KempCourse_data_organised/bids/'

## Step 1+2: Move and rename all files into a machine readable and BIDS-ready directory structure


#### ONCE ONLY: copy all raw data, with exceptions, to new directory (leave raw data untouched!)


In [15]:
# Ignore files:
# - with 'FSL', this is a proxy for the FSL '.nii' file
# - with spaces (this is a proxy for the xml/rec files),
# - the PAR/RECs of T1w file
src = data_dir
dst = new_org_data_dir
ignore_pattern = shutil.ignore_patterns('*FSL*',
                                        '* *',
                                        '*T1W*.PAR',
                                        '*T1W*.REC')
shutil.copytree(src, dst, ignore=ignore_pattern)

'/Users/jheunis/Desktop/KempCourse_data_organised/new/'

#### ONCE ONLY: Rename files such that bidsify can read separate tags and convert correctly to BIDS format
See filenames in: "/Users/jheunis/Documents/PYTHON/rtme-fMRI/bidsify_test_2709/sub-01")


In [16]:
# This means removing the 'SENSE' text from the T1w file such that files with SENSE are all recognised as BOLD
os.chdir(new_org_data_dir)
t1w = glob.glob('*/*T1W*',recursive=True)
for fn in t1w:
    os.rename(fn, fn.replace('_SENSE', ''))

## Step 3: Convert image data to BIDS

Using `bidsify`

See `me_workflow.ipynb`


In [17]:
!cd /Users/jheunis/Desktop/KempCourse_data_organised/new && bidsify -c /Users/jheunis/Desktop/KempCourse_data_organised/config.yml

Setting output-dir to /Users/jheunis/Desktop/KempCourse_data_organised/bids
Running bidsify with the following arguments:
	 directory=/Users/jheunis/Desktop/KempCourse_data_organised/new 
	 config=/Users/jheunis/Desktop/KempCourse_data_organised/config.yml 
	 out_dir=/Users/jheunis/Desktop/KempCourse_data_organised/bids 
	 validate=False

200221-11:50:42,371 nipype.utils INFO:
	 Running nipype version 1.2.3 (latest: 1.4.2)
Converting data from sub-111 ...
Compression will be faster with 'pigz' installed http://macappstore.org/pigz/
Chris Rorden's dcm2niiX version v1.0.20190410  Clang9.0.0 (64-bit MacOS)
Done reading PAR header version 4.2, with 21420 slices
Philips Scaling Values RS:RI:SS = 0.737:0:0.00218356 (see PMC3998685)
Convert 1 DICOM as /Users/jheunis/Desktop/KempCourse_data_organised/bids/sub-111/NEUFEP-ME-111_NEUFEP-ME-111_NEUFEP-ME-motor1_SENSE_3_1_echo-1 (64x64x34x210)
Philips Scaling Values RS:RI:SS = 0.737:0:0.00218356 (see PMC3998685)
Convert 1 DICOM as /Users/jheunis/De

## Next steps

Once the above steps are finished:
1. Head over to Matlab and run the `fmrwhy_workflow_fingertapping.m` script